In [1]:
import sympy
from sympy import Rational
import numpy as np
import pandas as pd
import math
from gurobipy import *

# Read the data
train_info = pd.read_csv('train_info_multi_class_multi_origin0607.csv') # Train timetable data 

# Input parameters
# Initial values of preference parameters
alpha_value_initial = 0.01
beta_value_initial = 0.01
gamma_value_initial = 0.01
g0_value_initial = 0.01

s = 0.0002 # Neighborhood radius value for sensitivity analysis
w = 1.618 # trial and error scale

di = len(train_info['TYPE']) #Number of train services
station_interval = list(range(151019789, 151019778, -1)) # Codes for sequential train running intervels from Tuqiao station to Sihui Dong station
origins_input = [151019789, 151019787, 151019785, 151019784] # Station codes of multiple origins
origins_interval_num = {151019789:11, 151019787:9, 151019785:7, 151019784:6} # Number of train running intervels of multiple origins
userclass_input = {origins_input[0]:[0, 1, 2, 3, 4], origins_input[1]:[0, 1, 2, 3, 4, 5], origins_input[2]:[0, 1, 2, 3, 4], origins_input[3]:[0, 1, 2, 3, 4, 5]} # Number of commuter class of multiple origins 

upper_parameters_record = sympy.Matrix([alpha_value_initial, beta_value_initial, gamma_value_initial, g0_value_initial]) # Upper level decision variable value record

upper_obj_record = [] # Upper objective value record
indicator_record = [] # Convergence criterion value record (δ)
upper_obj_sensitivity_record = [] # Upper objective sensitivity value record

# Train choice sets for multi-class commuters from different stations
train_choice_set_input = {}
train_choice_set_input[origins_input[0]] = {0:list(range(14,32)), 1:list(range(21,47)), 2:list(range(19,42)), 3:list(range(26,47)), 4:list(range(8,32))}
train_choice_set_input[origins_input[1]] = {0:list(range(24,42)), 1:list(range(14,32)), 2:list(range(16,35)), 3:list(range(26,47)), 4:list(range(5,27)), 5:list(range(24,45))}
train_choice_set_input[origins_input[2]] = {0:list(range(29,47)), 1:list(range(4,27)), 2:list(range(14,32)), 3:list(range(16,35)), 4:list(range(19,45))}
train_choice_set_input[origins_input[3]] = {0:list(range(11,30)), 1:list(range(19,35)), 2:list(range(24,42)), 3:list(range(16,30)), 4:list(range(26,47)), 5:list(range(3,25))}

# Train choice sets for multi-class commuters from different stations
demand_input = {}
demand_input[origins_input[0]] = {0: 155, 1: 81, 2: 112, 3: 53, 4: 95}
demand_input[origins_input[1]] = {0: 60, 1: 134, 2: 100, 3: 71, 4: 109, 5: 56}
demand_input[origins_input[2]] = {0: 57, 1: 91, 2: 100, 3: 87, 4: 95}
demand_input[origins_input[3]] = {0: 74, 1: 125, 2: 113, 3: 98, 4: 81, 5: 60}

# Desired arrival times for multi-class commuters from different stations
desired_arrival_time_input = {}
desired_arrival_time_input[origins_input[0]] = {0:30900, 1:33300, 2:32100, 3:34500, 4:29700}
desired_arrival_time_input[origins_input[1]] = {0:32700, 1:30900, 2:32100, 3:34500, 4:29700, 5:33300}
desired_arrival_time_input[origins_input[2]] = {0:34500, 1:29700, 2:30900, 3:31500, 4:32700}
desired_arrival_time_input[origins_input[3]] = {0:30300, 1:31500, 2:32700, 3:30900, 4:34500, 5:29100}

# Commuters' travel times for different trains at different stations
T = {}
for o in origins_input:
    for j in train_info['TYPE']:
        T[(o,j)] = train_info[train_info['TYPE'] == j]['TRIP_TIME_average_%d'%o].iloc[0]
        
# Train running time between adjacent stations 
Tv = {151019789: 110, 151019788: 130, 151019787: 140, 151019786: 120, 151019785: 160, 151019784: 180, 151019783: 180, 151019782: 180, 151019781: 200, 151019780: 170, 151019779: 110}

# Background passenger flows along the Batong Line during the morning peak period
n0 = {}
for j in train_info['TYPE']:
    for i in station_interval:
        n0[(j,i)] = train_info[train_info['TYPE'] == j]['%d'%i].values[0]

# Schedule delay time arriving early and Schedule delay time arriving late for multi-class commuters from different stations
TE = {}
TL = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            TE[(o,i,j)] = max(desired_arrival_time_input[o][i] - train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0], 0)
            TL[(o,i,j)] = max(train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0] - desired_arrival_time_input[o][i], 0)

In [2]:
# Definition of symbols for matrix calculation
# Symbols representing preference parameters
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
g0 = sympy.Symbol('g0')

lamb = sympy.Symbol('lamb') # Symbol for step size λ
no_num = sympy.Symbol('no_num') # Symbol for placeholder

# Symbols for departure rates
nis = {}
nis_keys = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis[(o,i,j)] = sympy.Symbol('n%d,%d,%d'%(o,i,j))
            nis_keys.append((o,i,j))
            
# Symbols for lagrange multiplier associated with constraint (10b)
miu = {}
for o in origins_input:
    for i in userclass_input[o]:
        miu[(o,i)] = sympy.Symbol('u%d,%d'%(o,i))

# Observed departure rates
y_observed_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            y_observed_li.append(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)][j-1])
y_observed = sympy.Matrix(y_observed_li)        
        
y_observed_dic = {}
for o in origins_input:
    y_observed_dic[o] = {}
    for i in userclass_input[o]:
        y_observed_dic[o][i] = sympy.Matrix(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)])

# Values of preference parameters
upper_parameters_value = {}
upper_parameters_value[alpha] = alpha_value_initial
upper_parameters_value[beta] = beta_value_initial
upper_parameters_value[gamma] = gamma_value_initial
upper_parameters_value[g0] = g0_value_initial

In [3]:
# In-vehicle commuter flow aggregations
flow_nis_aggregate = sympy.zeros(di,len(station_interval))
flow_nis_aggregate_dic = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] = flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(nis[(o,i,j)], (1, origins_interval_num[o])))

for j in train_info['TYPE']:
    for i in station_interval:
        flow_nis_aggregate_dic[(j, i)] = flow_nis_aggregate[(j - 1),(151019789 - i)]  

# In-vehicle crowding cost calculation
def invehicle_crowding_cost(origin, train, Tv_dic, n0_dic, flow_dic, g0_para):
    crowding_cost = 0
    for i in range(origin, 151019778, -1):
        crowding_cost = crowding_cost + g0_para * Tv_dic[i] * (flow_dic[(train, i)] + n0_dic[(train, i)] - 256) / 275
    return crowding_cost

In [4]:
# Derivatives of lagrange functions with respect to n
Ln = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            Ln.append((sympy.log(nis[(o,i,j)]) + 1) + invehicle_crowding_cost(o, j, Tv, n0, flow_nis_aggregate_dic, g0) + alpha * T[(o,j)] + beta * TE[(o,i,j)] + gamma * TL[(o,i,j)] - miu[(o,i)])            
            
Noi = []
for o in origins_input:
    for i in userclass_input[o]:
        noi = 0
        for j in train_choice_set_input[o][i]:
            noi = noi + nis[(o,i,j)]
        Noi.append(noi) 

# Block matrix B11
A_li = []
for num in range(0,len(Ln)):
    ln = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                ln.append(sympy.diff(Ln[num],nis[(o,i,j)]))
    A_li.append(ln)
    
A = sympy.Matrix(A_li)

# Block matrix B12
B_li = []
for num in range(0,len(Ln)):
    lu = []
    for o in origins_input:
        for i in userclass_input[o]:
            lu.append(sympy.diff(Ln[num],miu[(o,i)]))
    B_li.append(lu)
    
B = sympy.Matrix(B_li)

# Block matrix B21
C_li = []
for num in range(0,len(Noi)):
    nn = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nn.append(sympy.diff(Noi[num],nis[(o,i,j)]))
    C_li.append(nn)  
    
C = sympy.Matrix(C_li)

# Block matrix B21
D_li = []
for num in range(0,len(Noi)):
    nu = []
    for o in origins_input:
        for i in userclass_input[o]:
            nu.append(sympy.diff(Noi[num],miu[(o,i)]))
    D_li.append(nu)        

D = sympy.Matrix(D_li) 

# Block matrix N
N_li = []
for num in range(0,len(Ln)):
    lp = []
    lp.append(sympy.diff(Ln[num],alpha))
    lp.append(sympy.diff(Ln[num],beta))
    lp.append(sympy.diff(Ln[num],gamma))
    lp.append(sympy.diff(Ln[num],g0))
    N_li.append(lp)
    
N = sympy.Matrix(N_li)

epoch = 0
indicator = max(abs(upper_parameters_record[:,upper_parameters_record.shape[1] - 1] - upper_parameters_record[:,upper_parameters_record.shape[1] - 2]))

In [5]:
# Generalized travel cost of train services for multi-class commuters
def train_cost(flow_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_dic = {}
    for o in origins_input:
        for i in userclass[o]:
            for j in train_choice_set[o][i]:           
                train_cost_dic[(o, i, j)] = invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]

    return train_cost_dic

# Logit-based commuting flow loading
def logit(origin, user, train_cost_dic, demand, choice_set):
    
    cost_exp_li = []
    for j in train_info['TYPE']:
        if j in choice_set:
            cost_exp_li.append(sympy.exp( - train_cost_dic[(origin, user, j)]))
        else:
            cost_exp_li.append(0)
            
    flow = (sympy.Matrix(cost_exp_li) / np.sum(cost_exp_li)) * demand 
    return flow

# Logit-based commuting flow loading of multi-class commuters
def multiorigin_multiclass_logit(userclass, train_cost_dic, demand_dic, train_choice_set):
    
    flow_multiorigin_multiclass_aggregate = sympy.zeros(di,len(station_interval))
    flow_multiorigin_multiclass_dic = {}
    flow_multiorigin_multiclass_aggregate_dic = {}
    
    for o in origins_input:
        flow_multiorigin_multiclass_dic[o] = {}
        for i in userclass[o]:
            flow_multiorigin_multiclass_dic[o][i] = logit(o, i, train_cost_dic, demand_dic[o][i], train_choice_set[o][i])
            flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] = flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multiorigin_multiclass_dic[o][i], (1, origins_interval_num[o])))
    
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_multiorigin_multiclass_aggregate_dic[(j, i)] = flow_multiorigin_multiclass_aggregate[(j - 1),(151019789 - i)]
    
    return flow_multiorigin_multiclass_aggregate, flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic

# Generalized stochastic travel cost of train services for multi-class commuters
def train_cost_stochastic(flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_stochastic_dic = {}
    for o in origins_input:
        train_cost_stochastic_dic[o] = {}
        for i in userclass[o]:
            flow_origin_class_dic = dict(zip(train_info['TYPE'], flow_multiorigin_multiclass_dic[o][i]))
            train_cost_stochastic_dic[o][i] = {}
            for j in train_info['TYPE']:  
                if j in train_choice_set[o][i]:
                    train_cost_stochastic_dic[o][i][j] = (sympy.log(flow_origin_class_dic[j])+ 1) + invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_multiorigin_multiclass_aggregate_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]
                else:
                    train_cost_stochastic_dic[o][i][j] = no_num
                    
    return train_cost_stochastic_dic

# Bisection method to determine the step size
def binary_search(equation):
    a = 0
    b = 1
    fa = equation.evalf(subs = {lamb:a})
    fb = equation.evalf(subs = {lamb:b})
    while a <= b:
        mid = (a + b) / 2
        fm = equation.evalf(subs = {lamb:mid})
        if abs(fm) < 0.1:
            break
            
        if fa * fm < 0:
            a = a
            b = mid
            fb = equation.evalf(subs = {lamb:b})
            
        elif fb * fm < 0:
            a = mid
            b = b
            fa = equation.evalf(subs = {lamb:a})
            
    return mid

In [6]:
# Solve the rail corridor commuting equilibrium model(convex combination iteration algorithm)
def train_flow_assignment(userclass, train_choice_set, multi_demand, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    n = 0
    indicator_am = 1
    flow_dic_initial = {}
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_dic_initial[(j, i)] = 0
    cost_dic = train_cost(flow_dic_initial, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    flow_aggregate_x_new, flow_multi_dic_x_new, flow_aggregate_dic_x_new = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
    
    while (indicator_am > 0.0001) or (n <= 20):
    
        flow_multi_dic_x = flow_multi_dic_x_new
        flow_aggregate_dic_x = flow_aggregate_dic_x_new

        cost_dic = train_cost(flow_aggregate_dic_x, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        flow_aggregate_y, flow_multi_dic_y, flow_aggregate_dic_y = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set)
        
        # Auxiliary flow calculation
        flow_aggregate_aux = sympy.zeros(di,len(station_interval))
        flow_multi_dic_aux = {}
        flow_aggregate_dic_aux = {}
        
        for o in origins_input:
            flow_multi_dic_aux[o] = {}
            for i in userclass[o]:
                flow_multi_dic_aux[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb
                flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] = flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_aux[o][i], (1, origins_interval_num[o])))

        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_aux[(j, i)] = flow_aggregate_aux[(j - 1),(151019789 - i)]
                
        # Calculate the step size in this iteration
        train_cost_stochastic_aux_dic = train_cost_stochastic(flow_multi_dic_aux, flow_aggregate_dic_aux, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        equation_lamb = 0
        for o in origins_input:
            for i in userclass[o]:
                equation_lamb = equation_lamb + ((flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]).T * sympy.Matrix(list(train_cost_stochastic_aux_dic[o][i].values())))[0,0]

        lamb_value_df = binary_search(equation_lamb)
    
        # Determine the new iteration starting point
        flow_multi_dic_x_new = {}
        flow_aggregate_dic_x_new = {}
        flow_aggregate_x_new = sympy.zeros(di,len(station_interval))
        for o in origins_input:
            flow_multi_dic_x_new[o] = {}
            for i in userclass[o]:
                flow_multi_dic_x_new[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb_value_df
                flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] = flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_x_new[o][i], (1, origins_interval_num[o])))
    
        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_x_new[(j, i)] = flow_aggregate_x_new[(j - 1),(151019789 - i)]
        
        # Convergence criterion
        numerator = 0
        denominator = 0
        for o in origins_input:
            for i in userclass[o]:
                numerator = numerator + ((flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]).T * (flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]))[0,0]
                denominator = denominator + (sympy.ones(1,di) * flow_multi_dic_x[o][i])[0,0]
        
        indicator_am = (numerator ** 0.5) / denominator
        n = n + 1
        #print(indicator_am)
        #print(n)
        if (indicator_am <= 0.00001) or (n >= 20):
            break
    print(indicator_am)
    print(n)
    
    cost_dic_x_new = train_cost(flow_aggregate_dic_x_new, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    
    return flow_multi_dic_x_new, flow_aggregate_x_new, flow_aggregate_dic_x_new, cost_dic_x_new

In [ ]:
while (abs(indicator) >= 0.00000001) or (epoch == 0):
    
    epoch = epoch + 1
    flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0])
    
    # Departure rates of multi-class commuters of multiple origins
    nis_value_dic = {}
    nis_value_li = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
                nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         
    nis_value_matrix = sympy.Matrix(nis_value_li)
    
    # Upper level objective value record
    upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
    print(f'Upper level objective value-equilibrium:{upper_obj_record[-1]}')
    
    # Set of values of symbols
    matrix_parameters_value_all = {}
    matrix_parameters_value_all.update(nis_value_dic)
    matrix_parameters_value_all.update(upper_parameters_value)
    
    # Values of matrixes
    A_value = np.matrix(A.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    B_value = np.matrix(B.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    C_value = np.matrix(C.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    D_value = np.matrix(D.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
    N_value = np.matrix(N.evalf(subs = matrix_parameters_value_all)).astype(np.float64)

    # Gradient calculation
    I11_value = A_value.I + A_value.I * B_value * (D_value - C_value * A_value.I * B_value).I * C_value * A_value.I
    Grad = - I11_value * N_value    
    
    # Input parameters for NRMFD algorithm
    H = {}
    H[1] = 0.1 * 1000000000
    H[2] = 1 * 1000000000
    H[3] = 1 * 1000000000
    H[4] = 10 * 1000000000
 
    sigma = 4 #2~10
    
    Fn = ((nis_value_matrix - y_observed) * 2).T # The derivative of F with respect to n 
    Fe = dict(zip(list(range(1,5)), Fn * Grad)) # The derivative of F with respect to preference paramters
    
    # The Convex quadratic optimization problem to determine a descent direction
    NRMFD_model = Model("NRMFD")    
    
    #variables
    d = {}
    for i in range(1,5):
        d[i] = NRMFD_model.addVar(lb = - GRB.INFINITY, name = "direction%s" % i)

    theta = NRMFD_model.addVar(lb = - GRB.INFINITY, name = "theta")
    NRMFD_model.update()

    #objective
    NRMFD_model.setObjective(theta + 0.5 * sigma * (H[1] * (d[1])**2 + H[2] * (d[2])**2 + H[3] * (d[3])**2 + H[4] * (d[4])**2), GRB.MINIMIZE)  
    
    #constraints
    NRMFD_model.addConstr(quicksum(Fe[i] * d[i] for i in range(1,5)) <= theta, name = "NRMFD_inequation1")

    NRMFD_model.addConstr( - upper_parameters_value[alpha] - d[1] <= theta, name = "NRMFD_inequation2")
    NRMFD_model.addConstr( - upper_parameters_value[beta] - d[2] <= theta, name = "NRMFD_inequation3")
    NRMFD_model.addConstr( - upper_parameters_value[gamma] - d[3] <= theta, name = "NRMFD_inequation4")
    NRMFD_model.addConstr( - upper_parameters_value[g0] - d[4] <= theta, name = "NRMFD_inequation5")

    NRMFD_model.optimize()
    NRMFD_model.write('NRMFD_test.lp')
    
    # The descent direction of upper-level decision variables
    upper_level_direction = sympy.Matrix([n.x for n in NRMFD_model.getVars() if n.varName[0:9] == 'direction'])
    theta_value = [n.x for n in NRMFD_model.getVars() if n.varName == 'theta'][0]
    print(f'upper_level_direction:{upper_level_direction}')
    print(f'theta:{theta_value}')
    
    # The step size of upper-level decision variables
    step = s / max(upper_level_direction.applyfunc(sympy.Abs))
    print(step * upper_level_direction)
    
    y = nis_value_matrix + Grad * step * upper_level_direction
    upper_model_objvalue = sympy.simplify(((y_observed - y).T * (y_observed - y))[0,0])

    u_alpha = upper_parameters_value[alpha] + step * upper_level_direction[0]
    u_beta = upper_parameters_value[beta] + step * upper_level_direction[1]
    u_gamma = upper_parameters_value[gamma] + step * upper_level_direction[2]
    u_g0 = upper_parameters_value[g0] + step * upper_level_direction[3]
    
    while upper_model_objvalue > upper_obj_record[-1] or u_alpha < 0 or u_beta < 0 or u_gamma < 0 or u_g0 < 0:
        step = step * (1 / w)
        print(f'step_reduce:{step}')
        print(step * upper_level_direction)
        
        y = nis_value_matrix + Grad * step * upper_level_direction
        upper_model_objvalue = sympy.simplify(((y_observed - y).T * (y_observed - y))[0,0])
        
        u_alpha = upper_parameters_value[alpha] + step * upper_level_direction[0]
        u_beta = upper_parameters_value[beta] + step * upper_level_direction[1]
        u_gamma = upper_parameters_value[gamma] + step * upper_level_direction[2]
        u_g0 = upper_parameters_value[g0] + step * upper_level_direction[3]
        
    #  upper-level decision variable values update
    upper_parameters_value[alpha] = upper_parameters_value[alpha] + step * upper_level_direction[0]
    upper_parameters_value[beta] = upper_parameters_value[beta] + step * upper_level_direction[1]
    upper_parameters_value[gamma] = upper_parameters_value[gamma] + step * upper_level_direction[2]
    upper_parameters_value[g0] = upper_parameters_value[g0] + step * upper_level_direction[3]

    print(upper_parameters_value)
        
    # Calculate the indicator of convergence criterionc of upper level model
    upper_parameters_record = upper_parameters_record.col_insert(upper_parameters_record.shape[1], sympy.Matrix([upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0]]))

    if epoch > 1:
        indicator = upper_obj_record[-1] - upper_obj_record[-2]
        indicator_record.append(indicator)
        print(f'indicator:{indicator}')
    else:
        indicator = 1
    
    upper_obj_sensitivity_record.append(upper_model_objvalue)
    print(f'Upper level objective value-sensitivity analysis:{upper_model_objvalue}')
    print(f'Parameter optimization result：{upper_parameters_value}')
    print(f'Epoch:{epoch}'))